# Q - Learning for the TSP 

Consider the Travelling Salesman Problem (TSP) with n cities. [...] Here, an instance of TSP will be fully described by a matrix $D$ of size $n \times n$, where $n$ is the number of cities and $D[i,j]$ is the distance between city $i$ and city $j$.

Here is a method to find a solution (of good quality but not always optimal) using the Q learning method. An agent will be travelling trough the cities, going through each city exactly one time, receiving rewards at each stop and when he finishes a circuit. Over the iteration, the agent will explore the space ( [explain $\epsilon$ - greedy] ) and learn the Q matrix which will in the end provide a way to compute a good solution.

The agent will want to maximize his reward. The reward matrix will be such that the agents is reward $ - D[i,j]$ when he travels from city $i$ to city $j$ and $G = 100$ (arbitrary maybe change it ?) when he finishes a circuit. Since we don't allow the agent to go through the same city twice, we will set $D[i,i] = \infty $.

The $Q$ matrix should contain a cell for every pair $state \times action$, here the state and the action can both be represented by the cities. This matrix will be "dynamic" because it will change after each action (because once a city is visited, it is no longer part of the set of possible future actions). This is just for ease of implementation. [Explain a little bit]. 

One iteration of the algorithm will consist in the agent building one path through the cities. During one iteration, we will have a parameter $\epsilon$ controlling the $\epsilon-greedy$ search, that is every time the agent finds himself in city $i$, he will go to a random city (among those not yet visited) with probability $\epsilon$ and go the city that yield the highest future reward according to the Q matrix with probability $1-\epsilon$. After each iteration, $\epsilon$ will be decreased such that $\epsilon_{t+1} = \epsilon_{t} \cdot (1-\lambda)$, with $\lambda \in (0,1)$

Each time the agent arrives to a city, we update one cell of the Q matrix according to :

$Q^{new}(s_{t},a_{t})\leftarrow \underbrace {Q(s_{t},a_{t})} _{\text{old value}}+\underbrace {\alpha } _{\text{learning rate}}\cdot \overbrace {{\bigg (}\underbrace {\underbrace {r_{t}} _{\text{reward}}+\underbrace {\gamma } _{\text{discount factor}}\cdot \underbrace {\max _{a}Q(s_{t+1},a)} _{\text{estimate of optimal future value}}} _{\text{new value (temporal difference target)}}-\underbrace {Q(s_{t},a_{t})} _{\text{old value}}{\bigg )}} ^{\text{temporal difference}}$

*(formula from wikipedia)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import random
import networkx as nx

In [24]:
class QLearnTSP:
    def __init__(self, dist):
        self.len_path = dist.shape[0]
        self.dist = dist
        #reward matrix, and Q matrix under the form of dictionnaries
        #--------- should be optimized --------- #
        start = 0 
        end = 0 
        rewards = {}
        q = {}
        states = ["start"] + [i for i in range(1, self.len_path)] + ["end"]
        for s in states:
            for n in states:
                if s == n: 
                    q[(s,n)] = -np.inf
                    continue
                q[(s,n)]=0
                s_i, n_i = s, n
                if type(s) == str:
                    s_i = 0
                if type(n) == str:
                    n_i = 0
                if s_i == n_i: 
                    continue    

                rewards[(s_i, n_i)] = -dist[s_i, n_i]
                if n_i == 0:
                    rewards[(s_i, n_i)] = 100 - dist[s_i, n_i]  
        self.rewards = rewards
        self.q = q
    
    

    def follow_path(self, q_temp):
        path = ['start']
        while 'end' not in path:
            possible_next = set([i for i in range(1, self.len_path)]) - set(path)
            if possible_next == set():
                path.append('end')
                continue
            next_state = list(possible_next)[0]
            r = q_temp[(path[-1], next_state)]
            for ele in possible_next:
                if q_temp[(path[-1], ele)] > r:
                    next_state = ele
                    r = q_temp[(path[-1], ele)]
            path.append(next_state)
        return [0 if x in ['start', 'end'] else x for x in path]

    
    
    def get_cost(self, path):
        c = 0
        for i in range(1, len(path)):
            c += self.dist[path[i-1], path[i]]
        return c

    def qlearning(self, epsilon, gamma, lr, lbda, epochs = 100):
        start = 0
        eps = epsilon
        q_learn = self.q.copy()
        for t in range(epochs):
            if t%(epochs/10) == 0:
                print("Iteration :", t, "||", "epsilon = ", round(eps * (1-lbda), 2), "||", "Current cost :", self.get_cost(self.follow_path(q_learn)))

            current_state = start
            current_path = [start]
            eps = eps * (1-lbda)
            while len(current_path) < self.len_path+1:
                if len(current_path) == self.len_path:
                    current_path.append(0)
                    #do 

                    #
                    continue
                current_state = current_path[-1]
                possible_next = set([i for i in range(1, self.len_path)]) - set(current_path)
                u = np.random.random()
                if u < eps:
                    next_state = np.random.choice(list(possible_next))
                else:
                    key_next = [(current_state, n) for n in possible_next]
                    next_state = list(possible_next)[0]
                    r = self.rewards[(current_state, next_state)]
                    for e in key_next:
                        if self.rewards[e] > r:
                            next_state = e[1]
                            r = self.rewards[e]
                current_path.append(next_state)
                #updating Q
                c_s, n_s, c_s_i, n_s_i = current_state, next_state, current_state, next_state
                if current_state == 0:
                    c_s_i = 'start'
                if next_state == 0:
                    n_s_i = 'end'
                possible_next = set([i for i in range(1, self.len_path)]) - set(current_path) - set([next_state])
                if possible_next == set():
                    continue

                max_r_n = list(possible_next)[0]
                m_r = q_learn[(c_s_i, n_s_i)]
                for ele in possible_next.union(set(["end"])):
                    if q_learn[(c_s_i, ele)] > m_r:
                        max_r_n = ele
                        m_r = q_learn[(c_s_i, ele)] 

                q_learn[(c_s_i, n_s_i)] = q_learn[(c_s_i, n_s_i)] + lr * ((self.rewards[(c_s, n_s)] + gamma * m_r - q_learn[(c_s_i, n_s_i)]))
        return q_learn
            

In [20]:
dist_matrix_15 = np.loadtxt("tsp_15_291.txt") #15 cities and min cost = 291
dist_matrix_26 = np.loadtxt("tsp_26_937.txt")
dist_matrix_17 = np.loadtxt("tsp_17_2085.txt")
dist_matrix_42 = np.loadtxt("tsp_42_699.txt")
dist_matrix_48 = np.loadtxt("tsp_48_33523.txt")

In [25]:
solver = QLearnTSP(dist_matrix_15)

In [26]:
Q_15 = solver.qlearning(1, 0.88, 0.1, 0.0001, 4000)

Iteration : 0 || epsilon =  1.0 || Current cost : 817.0
Iteration : 400 || epsilon =  0.96 || Current cost : 295.0
Iteration : 800 || epsilon =  0.92 || Current cost : 291.0
Iteration : 1200 || epsilon =  0.89 || Current cost : 291.0
Iteration : 1600 || epsilon =  0.85 || Current cost : 291.0
Iteration : 2000 || epsilon =  0.82 || Current cost : 291.0
Iteration : 2400 || epsilon =  0.79 || Current cost : 291.0
Iteration : 2800 || epsilon =  0.76 || Current cost : 291.0
Iteration : 3200 || epsilon =  0.73 || Current cost : 291.0
Iteration : 3600 || epsilon =  0.7 || Current cost : 291.0


In [27]:
solver = QLearnTSP(dist_matrix_17)

In [29]:
Q_17 = solver.qlearning(1, 0.88, 0.1, 0.0001, 10000)

Iteration : 0 || epsilon =  1.0 || Current cost : 4517.0
Iteration : 1000 || epsilon =  0.9 || Current cost : 2199.0
Iteration : 2000 || epsilon =  0.82 || Current cost : 2199.0
Iteration : 3000 || epsilon =  0.74 || Current cost : 2199.0
Iteration : 4000 || epsilon =  0.67 || Current cost : 2187.0
Iteration : 5000 || epsilon =  0.61 || Current cost : 2187.0
Iteration : 6000 || epsilon =  0.55 || Current cost : 2187.0
Iteration : 7000 || epsilon =  0.5 || Current cost : 2187.0
Iteration : 8000 || epsilon =  0.45 || Current cost : 2187.0
Iteration : 9000 || epsilon =  0.41 || Current cost : 2187.0


In [30]:
solver = QLearnTSP(dist_matrix_26)

In [35]:
Q_26 = solver.qlearning(1, 0.88, 0.2, 0.001, 10000)

Iteration : 0 || epsilon =  1.0 || Current cost : 1159.0
Iteration : 1000 || epsilon =  0.37 || Current cost : 1050.0
Iteration : 2000 || epsilon =  0.14 || Current cost : 1050.0
Iteration : 3000 || epsilon =  0.05 || Current cost : 1050.0
Iteration : 4000 || epsilon =  0.02 || Current cost : 1050.0
Iteration : 5000 || epsilon =  0.01 || Current cost : 1050.0
Iteration : 6000 || epsilon =  0.0 || Current cost : 1050.0
Iteration : 7000 || epsilon =  0.0 || Current cost : 1050.0
Iteration : 8000 || epsilon =  0.0 || Current cost : 1050.0
Iteration : 9000 || epsilon =  0.0 || Current cost : 1050.0


In [37]:
solver = QLearnTSP(dist_matrix_42)

In [45]:
Q_42 = solver.qlearning(1, 0.8, 0.1, 0.0001, 10000)

Iteration : 0 || epsilon =  1.0 || Current cost : 962.0
Iteration : 1000 || epsilon =  0.9 || Current cost : 943.0
Iteration : 2000 || epsilon =  0.82 || Current cost : 954.0
Iteration : 3000 || epsilon =  0.74 || Current cost : 955.0
Iteration : 4000 || epsilon =  0.67 || Current cost : 955.0
Iteration : 5000 || epsilon =  0.61 || Current cost : 955.0
Iteration : 6000 || epsilon =  0.55 || Current cost : 955.0
Iteration : 7000 || epsilon =  0.5 || Current cost : 955.0
Iteration : 8000 || epsilon =  0.45 || Current cost : 955.0
Iteration : 9000 || epsilon =  0.41 || Current cost : 955.0


In [46]:
solver = QLearnTSP(dist_matrix_48)

In [47]:
Q_48 = solver.qlearning(1, 0.8, 0.1, 0.0001, 4000)

Iteration : 0 || epsilon =  1.0 || Current cost : 151722.0
Iteration : 400 || epsilon =  0.96 || Current cost : 48117.0
Iteration : 800 || epsilon =  0.92 || Current cost : 41718.0
Iteration : 1200 || epsilon =  0.89 || Current cost : 40985.0
Iteration : 1600 || epsilon =  0.85 || Current cost : 40985.0
Iteration : 2000 || epsilon =  0.82 || Current cost : 40985.0
Iteration : 2400 || epsilon =  0.79 || Current cost : 40985.0
Iteration : 2800 || epsilon =  0.76 || Current cost : 40985.0
Iteration : 3200 || epsilon =  0.73 || Current cost : 40551.0
Iteration : 3600 || epsilon =  0.7 || Current cost : 40551.0
